In [12]:
import pandas as pd
from src.config import AGENCIAS, CLIENTES, COLAB_AGENCIA, COLABORADORES, CONTAS, PROPOSTA_CREDITO, TRANSACOES

tables = pd.read_csv(TRANSACOES)
display(tables)

,cod_transacao,num_conta,data_transacao,nome_transacao,valor_transacao
0,3100,256,2020-05-22 03:15:21 UTC,Saque,-130.0
1,28405,256,2021-06-29 11:52:38 UTC,Saque,-255.0
2,1776,512,2020-05-17 13:04:50 UTC,Saque,-250.0
3,14311,512,2020-09-06 02:55:34 UTC,Saque,-80.0
4,25012,512,2020-10-16 09:26:52 UTC,Saque,-475.0
...,...,...,...,...,...
71994,17007,764,2022-12-29 23:59:59.746608 UTC,Transferência entre CC - Crédito,82.3
71995,22320,764,2022-12-29 23:59:59.055028 UTC,Transferência entre CC - Crédito,414.8
71996,8697,509,2012-05-07 13:45:41 UTC,Transferência entre CC - Crédito,720.4
71997,11574,766,2015-09-19 05:13:07 UTC,Transferência entre CC - Crédito,1143.0


In [14]:
import pandas as pd

df_transacoes = pd.read_csv(TRANSACOES, encoding='utf-8')
df_proposta_credit = pd.read_csv(PROPOSTA_CREDITO, encoding='utf-8')

# Converter a coluna de data para datetime
df_transacoes["data_transacao"] = pd.to_datetime(df_transacoes["data_transacao"], errors="coerce").dt.tz_localize(None)

# Criar coluna de trimestre
df_transacoes["trimestre"] = df_transacoes["data_transacao"].dt.to_period("Q")

# Pegando apenas status de propostas aprovadas
df_aprovadas = df_proposta_credit[df_proposta_credit["status_proposta"] == "Aprovada"]

# Merge para unir o cod_transacao com cod_proposta
df_juncao = df_transacoes.merge(df_aprovadas, left_on="cod_transacao", right_on="cod_proposta", how="inner")

# Agrupar por trimestre e calcular a média de transações e volume movimentado
df_resultado = df_juncao.groupby("trimestre").agg(
    media_transacoes_aprovadas=("cod_transacao", "count"),
    media_valor_movimentado=("valor_transacao", "mean")
).reset_index()

# Identificar os trimestres com maiores valores
trimestre_mais_transacoes = df_resultado.loc[df_resultado["media_transacoes_aprovadas"].idxmax()]
trimestre_maior_volume = df_resultado.loc[df_resultado["media_valor_movimentado"].idxmax()]

display(df_resultado)

# Exibir resultados
print("Trimestre com mais transações aprovadas:", trimestre_mais_transacoes["trimestre"])
print("Média de transações aprovadas:", trimestre_mais_transacoes["media_transacoes_aprovadas"])

print("\nTrimestre com maior volume movimentado:", trimestre_maior_volume["trimestre"])
print("Média do volume movimentado:", trimestre_maior_volume["media_valor_movimentado"])

,trimestre,media_transacoes_aprovadas,media_valor_movimentado
0,2010Q1,2,760.080000
1,2010Q2,2,95.580000
2,2010Q3,1,-52.150000
3,2010Q4,2,41.655000
4,2011Q1,4,300.070000
5,2011Q2,6,954.856667
6,2011Q4,1,179.500000
7,2012Q1,2,83.450000
8,2012Q2,6,149.765000
9,2012Q3,5,157.232000


Trimestre com mais transações aprovadas: 2022Q4
Média de transações aprovadas: 26

Trimestre com maior volume movimentado: 2014Q4
Média do volume movimentado: 4302.95


In [4]:
import pandas as pd

df_transacoes = pd.read_csv(TRANSACOES, encoding='utf-8')
df_transacoes["data_transacao"] = pd.to_datetime(df_transacoes["data_transacao"], errors="coerce").dt.tz_localize(None)

df_transacoes["mes_nome"] = df_transacoes["data_transacao"].dt.strftime("%B")

# Verifica se contém "r"
df_transacoes["mes_tem_R"] = df_transacoes["mes_nome"].str.lower().str.contains("r")

# Contar a média de transações aprovadas por grupo (meses com "R" versus sem "R")
resultado_analise = df_transacoes.groupby("mes_tem_R").agg(
    media_transacoes=("cod_transacao", "count"),
    media_valor_movimentado=("valor_transacao", "mean")
).reset_index()

resultado_analise["mes_tem_R"] = resultado_analise["mes_tem_R"].map({True: "Meses com R", False: "Meses sem R"})

# Exibir resultado
display(resultado_analise)

,mes_tem_R,media_transacoes,media_valor_movimentado
0,Meses sem R,15558,383.537574
1,Meses com R,42539,422.161968


In [10]:
import pandas as pd

df_transacoes = pd.read_csv(TRANSACOES, encoding="utf-8")
df_transacoes["data_transacao"] = pd.to_datetime(df_transacoes["data_transacao"], errors="coerce").dt.tz_localize(None)

# Criar coluna para o ano da transação
df_transacoes["ano"] = df_transacoes["data_transacao"].dt.year

# Criar uma coluna que indica se o mês contém "R" no nome
df_transacoes["mes_nome"] = df_transacoes["data_transacao"].dt.strftime("%B")

# Verifica se contém "r"
df_transacoes["mes_tem_R"] = df_transacoes["mes_nome"].str.lower().str.contains("r")

# Agrupar por ano e pela presença de "R" no nome do mês
resultado_analise = df_transacoes.groupby(["ano", "mes_tem_R"]).agg(
    media_transacoes=("cod_transacao", "count"),
    media_valor_movimentado=("valor_transacao", "mean")
).reset_index()

# Mapeia os valores booleanos para legibilidade
resultado_analise["mes_tem_R"] = resultado_analise["mes_tem_R"].map({True: "Meses com R", False: "Meses sem R"})

# Ordena os anos em ordem crescente
resultado_analise = resultado_analise.sort_values(by="ano")

display(resultado_analise)

# Contar a média de transações aprovadas por grupo (meses com "R" versus sem "R")
resultado_total_analise = df_transacoes.groupby("mes_tem_R").agg(
    media_transacoes=("cod_transacao", "count"),
    media_valor_movimentado=("valor_transacao", "mean")
).reset_index()

resultado_total_analise["mes_tem_R"] = resultado_total_analise["mes_tem_R"].map({True: "Meses com R", False: "Meses sem R"})

print("TOTAL\n", resultado_total_analise)

,ano,mes_tem_R,media_transacoes,media_valor_movimentado
0,2010.0,Meses sem R,23,441.998261
1,2010.0,Meses com R,70,264.293429
2,2011.0,Meses sem R,169,1423.170414
3,2011.0,Meses com R,198,887.471162
4,2012.0,Meses sem R,238,1245.380378
5,2012.0,Meses com R,496,861.963427
6,2013.0,Meses sem R,428,212.997617
7,2013.0,Meses com R,821,925.089123
8,2014.0,Meses sem R,516,1092.825601
9,2014.0,Meses com R,986,511.166034


TOTAL
      mes_tem_R  media_transacoes  media_valor_movimentado
0  Meses sem R             15558               383.537574
1  Meses com R             42539               422.161968
